In [69]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [70]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [72]:
import pandas as pd
import os
import pickle
import torch
from torch import Tensor
from collections import defaultdict
from tqdm import tqdm
from datetime import datetime
from sentence_transformers import SentenceTransformer, util
from difflib import SequenceMatcher

In [73]:
#Change only this cell to compute the matches between any region and its tar. 
#Make sure that the file's name have the following form: "bando_cig_VENETO.csv" and "tar_ve_recourses.csv" containd in a folder called "Recourse_VENETO".

#Choose the desired  model, region, tar
region = "MOLISE"
tar = ["cb"] #write in a list all the tar of the chosen region (in alphabetical order)
model_name = "LaBSE"

dir_in = f"/content/drive/MyDrive/K-NearestNeighbors/Recourse/Recourse_{region}"

In [74]:
#load the file with all the tenders
filename = f"bando_cig_{region}.csv"
col_select = ["cig", "oggetto_gara", "importo_complessivo_gara", "oggetto_lotto", "importo_lotto", "sezione_regionale", "anno_pubblicazione", "cf_amministrazione_appaltante"]
tender_df = pd.read_csv(dir_in + os.sep + filename, delimiter = ";", header = 0, dtype = {"cig":object}, low_memory = False, usecols = col_select) # read the CSV and generate the DF of tenders

#adding from "aggiudicazioni_csv_clean.csv" the date of the award (i.e. data_aggiudicazione_definitiva )
filename = f"aggiudicazioni_csv_clean.csv"
col_select = ["cig", "data_aggiudicazione_definitiva"]

aggiudicazioni_df = pd.read_csv(f"/content/drive/MyDrive/K-NearestNeighbors/Recourse" + os.sep + filename, delimiter = ";", header = 0, low_memory = False, usecols = col_select) 
aggiudicazioni_df = aggiudicazioni_df.drop(aggiudicazioni_df[["cig"]][aggiudicazioni_df[["cig"]].duplicated(keep=False)].index).reset_index(drop=True) 


tender_df = pd.merge(tender_df, aggiudicazioni_df, how='left')

tender_df 

,cig,oggetto_gara,importo_complessivo_gara,oggetto_lotto,importo_lotto,cf_amministrazione_appaltante,sezione_regionale,anno_pubblicazione,data_aggiudicazione_definitiva
0,2713844EE6,FORNITURA SERVIZIO ASSISTENZA DOMICILIAR DISTR...,NaN,FORNITURA SERVIZIO ASSISTENZA DOMICILIARE DIST...,-1.00,01546900703,SEZIONE REGIONALE MOLISE,2007,NaN
1,2807606DC0,LAVORI DI MESSA IN SICUREZZA DEL PLESSO SCOLAS...,807294.75,LAVORI DI MESSA IN SICUREZZA DEL PLESSO SCOLAS...,807294.78,80000270944,SEZIONE REGIONALE MOLISE,2007,NaN
2,13768948DD,RIPARAZIONE CON MIGLIORAMENTO SISMICO FABBRICA...,97907.83,RIPARAZIONE CON MIGLIORAMENTO SISMICO FABBRICA...,97907.83,00054410709,SEZIONE REGIONALE MOLISE,2007,NaN
3,1512414383,MANUTENZIONE PARCOMETRI,26801.55,MANUTENZIONE PARCOMETRI,26801.55,01500620701,SEZIONE REGIONALE MOLISE,2007,NaN
4,1511304F7F,"RACCOLTA, TRASPORTO E SMALTIMENTO PERCOLATO DI...",30352.11,"RACCOLTA, TRASPORTO E SMALTIMENTO PERCOLATO DI...",30352.11,01500620701,SEZIONE REGIONALE MOLISE,2007,NaN
...,...,...,...,...,...,...,...,...,...
23620,953339459A,REALIZZAZIONE ASILO NIDO COMUNALE IN VIA MARCONI,4587.87,REALIZZAZIONE ASILO NIDO COMUNALE IN VIA MARCONI,4587.87,80002350702,SEZIONE REGIONALE MOLISE,2022,NaN
23621,9504202B98,CONTRATTO ISTITUZIONALE DI SVILUPPO PER IL MOL...,77582.34,CONTRATTO ISTITUZIONALE DI SVILUPPO PER IL MOL...,77582.34,80002910703,SEZIONE REGIONALE MOLISE,2022,NaN
23622,9567298013,GARA PER LA FORNITURA DI UN SISTEMA DI MONITOR...,150000.00,GARA PER LA FORNITURA DI UN SISTEMA DI MONITOR...,150000.00,01546900703,SEZIONE REGIONALE MOLISE,2022,NaN
23623,953029690C,SERVIZIO DI RACCOLTA DIFFERENZIATA DEI RIFIUTI...,506714.44,SERVIZIO DI RACCOLTA DIFFERENZIATA DEI RIFIUTI...,506714.44,00038870945,SEZIONE REGIONALE MOLISE,2022,NaN


In [75]:
#load the file with all the recourses of the selected region

filename = f"tar_{tar[0]}_recourses.csv"
recourse_df = pd.read_csv(dir_in + os.sep + filename, delimiter = ";", header = 0, low_memory = False, dtype = {"recourse_num":object, "recourse_id":object},on_bad_lines = "warn")
  
for i in range(1,len(tar)):
  filename = f"tar_{tar[i]}_recourses.csv"
  data = pd.read_csv(dir_in + os.sep + filename, delimiter = ";", header = 0, low_memory = False, dtype = {"recourse_num":object, "recourse_id":object},on_bad_lines = "warn")
  df = pd.DataFrame(data)
  recourse_df = pd.concat([recourse_df,df],axis=0)

recourse_df = recourse_df.dropna(subset=["recourse_object_df"]).reset_index(drop=True)
recourse_df = recourse_df.drop(recourse_df[recourse_df.recourse_year < tender_df[["anno_pubblicazione"]].min()[0]].index).reset_index(drop=True)


#adding from "iaj_recourses_recourses_ES_clean.csv" the feautre_id (i.e. cf_amministrazione_appaltante )
filename = f"iaj_recourses_recourses_ES_clean.csv"
col_select = ["court", "recourse_num", "feature_id"]

df = pd.read_csv(f"/content/drive/MyDrive/K-NearestNeighbors/Recourse" + os.sep + filename, delimiter = ";", header = 0, low_memory = False, dtype = {"recourse_num":object, "recourse_id":object},on_bad_lines = "warn", usecols = col_select) 
tars = df[df["court"] == f"tar_{tar[0]}"].reset_index(drop=True)

for i in range(1, len(tar)):
  df2 = df[df["court"] == f"tar_{tar[i]}"].reset_index(drop=True)
  tars = pd.concat([tars,df2],axis=0)

tars = tars.drop(tars[tars.duplicated(['recourse_num'], keep=False)].index).reset_index(drop=True) 

recourse_df = pd.merge(recourse_df, tars, how='left')
recourse_df = recourse_df.drop_duplicates(keep='first').reset_index(drop=True) #remove duplicates


recourse_df = recourse_df.rename(columns={'recourse_date': 'recourse_date_df'})



recourse_df

b'Skipping line 438: expected 6 fields, saw 11\n'


,court,recourse_num,recourse_year,recourse_id,recourse_date_df,recourse_object_df,feature_id
0,tar_cb,200800050,2008,00050,2008-02-06,AFFIDAMENTO IN CONCESSIONE DEL SERVIZIO DI ILL...,NaN
1,tar_cb,200700088,2007,00088,2007-03-08,GARA DI APPALTO PER LA PROGETTAZIONE DI ACQUED...,NaN
2,tar_cb,200700360,2007,00360,2007-08-28,ASSISTENZA DOMICILIARE ANZIANI - GARA PER SERV...,NaN
3,tar_cb,200700359,2007,00359,2007-08-27,GARA DI APPALTO PER AFFIDAMENTO ASSISTENZA AI ...,NaN
4,tar_cb,200800083,2008,00083,2008-02-20,GARA PER AFFIDAMENTO LAVORI DI SISTEMAZIONE TR...,NaN
...,...,...,...,...,...,...,...
409,tar_cb,202000313,2020,00313,2020-11-25,ACCESSO AGLI ATTI,NaN
410,tar_cb,201900235,2019,00235,2019-07-12,ANNULLAMENTO PROPOSTA DI AGGIUDICAZIONE NEI CO...,NaN
411,tar_cb,201900178,2019,00178,2019-05-30,PROVVEDIMENTO PROT. N. XXXXX DELLA PREFETTURA ...,NaN
412,tar_cb,202000228,2020,00228,2020-10-05,ACCORDO QUADRO QUADRIENNALE PER LESECUZIONE DI...,NaN


In [76]:
#Calculate the embeddings of tender_df["oggetto_gara"]
if not os.path.exists(dir_in + os.sep + f'embeddings_Bando_{region}_{model_name}.pkl'):
  
  model = SentenceTransformer(model_name)
  embeddings = model.encode(tender_df["oggetto_gara"], show_progress_bar=True, batch_size=128, convert_to_tensor=True)
  
  with open(dir_in + os.sep + f'embeddings_Bando_{region}_{model_name}.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

In [77]:
#Calculate the embeddings of recourse_df["recourse_object_df"]
if not os.path.exists(dir_in + os.sep + f'embeddings_recourse_tar_{region}_{model_name}.pkl'):

  model = SentenceTransformer(model_name)
  embeddings = model.encode(recourse_df["recourse_object_df"], show_progress_bar=True, batch_size=128, convert_to_tensor=True)
  
  with open(dir_in + os.sep + f'embeddings_recourse_tar_{region}_{model_name}.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

In [78]:
#Load the embeddings
with open(dir_in + os.sep + f'embeddings_Bando_{region}_{model_name}.pkl', 'rb') as f:
  embeddings_tenders = pickle.load(f)

with open(dir_in + os.sep + f'embeddings_recourse_tar_{region}_{model_name}.pkl', 'rb') as f:
  embeddings_recourses = pickle.load(f)

In [79]:
def cos_sim(a: Tensor, b: Tensor):
    """
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
    Returns: Matrix with result[i][j]  = cos_sim(a[i], b[j])
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))

In [80]:
def match_embeddings(embeddings_recourses: Tensor,
                           embeddings_tenders: Tensor,
                           recourse_df,
                           tender_df):
    """
    param embeddings_recourses: A tensor with the embeddings of "recourse_object_df"
    param embeddings_tenders: A tensor with the embeddings of "oggetto_gara"
    param recourse_df: pandas dataframe containing the attributes of the recourses
    param tender_df: pandas dataframe containing the attributes of the tenders
    Returns: pair dictionary where at each key (the index of the recourse) is associated the best matched tender together with their cosine similarity score.
             ties dictionary containing for each key (the index of the recourse) any possible tie with the best match.
    """       

    pairs = defaultdict(list)
    ties = defaultdict(list)

    for i in tqdm(range(0, len(embeddings_recourses))):
        scores = cos_sim(embeddings_recourses[i], embeddings_tenders) #cosine similarities between the i-th recourse and all the tenders

        top_score, top_idx = torch.topk(scores, len(scores[0]), dim=1, largest=True, sorted=True)
        top_score = top_score.cpu().tolist()[0]
        top_idx = top_idx.cpu().tolist()[0]

        n = 0

        #we consider only the tenders whose attributes (anno_pubblicazione, cf_amministrazione_appaltante, data_aggiudicazione_definitiva) are coherent with those of the recourses.
        if tender_df.at[top_idx[n], 'anno_pubblicazione'] <= recourse_df.at[i, 'recourse_year'] <= (tender_df.at[top_idx[n], 'anno_pubblicazione'] + 3) and \
           (pd.isnull(recourse_df.at[i, 'feature_id']) or tender_df.at[top_idx[n], 'cf_amministrazione_appaltante'] == recourse_df.at[i, 'feature_id']) and \
           (pd.isnull(tender_df.at[top_idx[n], 'data_aggiudicazione_definitiva']) or (datetime.strptime(recourse_df.at[i, 'recourse_date_df'], "%Y-%m-%d") - datetime.strptime(tender_df.at[top_idx[n], 'data_aggiudicazione_definitiva'], "%Y-%m-%d")).days <= 60):

            pairs[i] = [top_score[n], top_idx[n]] #save the best match
            n+=1
        else:
            while ((recourse_df.at[i, 'recourse_year'] < tender_df.at[top_idx[n], 'anno_pubblicazione'] or recourse_df.at[i, 'recourse_year'] > (tender_df.at[top_idx[n], 'anno_pubblicazione'] + 3) or not \
                   (pd.isnull(recourse_df.at[i, 'feature_id']) or tender_df.at[top_idx[n], 'cf_amministrazione_appaltante'] == recourse_df.at[i, 'feature_id'])) or not \
                   (pd.isnull(tender_df.at[top_idx[n], 'data_aggiudicazione_definitiva']) or (datetime.strptime(recourse_df.at[i, 'recourse_date_df'], "%Y-%m-%d") - datetime.strptime(tender_df.at[top_idx[n], 'data_aggiudicazione_definitiva'], "%Y-%m-%d")).days <= 60)) \
                   and top_score[n] > 0.30: 
                n += 1
            pairs[i] = [top_score[n], top_idx[n]] #save the best match
            n += 1

        #we look if there are ties (or almost ties, i.e with cosine similarity not smaller than 0.01 from the best match)     
        while pairs[i][0] - top_score[n] < 0.01  and top_score[n] > 0.30:

            if tender_df.at[top_idx[n], 'anno_pubblicazione'] <= recourse_df.at[i, 'recourse_year'] <= (tender_df.at[top_idx[n], 'anno_pubblicazione'] + 3) and \
                (pd.isnull(recourse_df.at[i, 'feature_id']) or tender_df.at[top_idx[n], 'cf_amministrazione_appaltante'] == recourse_df.at[i, 'feature_id']) and \
                (pd.isnull(tender_df.at[top_idx[n], 'data_aggiudicazione_definitiva']) or (datetime.strptime(recourse_df.at[i, 'recourse_date_df'], "%Y-%m-%d") - datetime.strptime(tender_df.at[top_idx[n], 'data_aggiudicazione_definitiva'], "%Y-%m-%d")).days <= 60):

                ties[i].append([top_score[n], top_idx[n]]) #save the ties
                n += 1
            else:
                while ((recourse_df.at[i, 'recourse_year'] < tender_df.at[top_idx[n], 'anno_pubblicazione'] or recourse_df.at[i, 'recourse_year'] > (tender_df.at[top_idx[n], 'anno_pubblicazione'] + 3) or not \
                       (pd.isnull(recourse_df.at[i, 'feature_id']) or tender_df.at[top_idx[n], 'cf_amministrazione_appaltante'] == recourse_df.at[i, 'feature_id'])) or not \
                       (pd.isnull(tender_df.at[top_idx[n], 'data_aggiudicazione_definitiva']) or (datetime.strptime(recourse_df.at[i, 'recourse_date_df'], "%Y-%m-%d") - datetime.strptime(tender_df.at[top_idx[n], 'data_aggiudicazione_definitiva'], "%Y-%m-%d")).days <= 60)) \
                       and top_score[n] > 0.30:
                    n += 1
                if pairs[i][0] - top_score[n] < 0.01:
                  ties[i].append([top_score[n], top_idx[n]]) #save the ties
                  n += 1

    return pairs, ties

In [81]:
matches, ties = match_embeddings(embeddings_recourses, embeddings_tenders, recourse_df, tender_df)

100%|██████████| 414/414 [00:02<00:00, 166.03it/s]


In [82]:
first10matches = {k: matches[k] for k in list(matches)[:10]}
first10matches

{0: [0.6119096279144287, 910],
 1: [0.44847920536994934, 804],
 2: [0.4809546172618866, 754],
 3: [0.4392017126083374, 416],
 4: [0.48175105452537537, 556],
 5: [0.5773910284042358, 596],
 6: [0.6180230379104614, 391],
 7: [0.4860016703605652, 819],
 8: [0.5319764614105225, 47],
 9: [0.6092954874038696, 3816]}

In [83]:
def get_best_match(query, corpus, step=4, flex=3, case_sensitive=False, verbose=False):
    """
    Creation of a "fuzzy" score, able to find the text of the query inside a larger corpus.
    param query : str
    param corpus : str
    param step (int) : Step size of first match-value scan through corpus.
    param flex (int) : Max. left/right substring position adjustment value.
    return: Best matching substring.
            Match ratio of best matching substring. 1 is perfect match.
    """

    def _match(a, b):
        """Compact alias for SequenceMatcher."""
        return SequenceMatcher(None, a, b).ratio()

    def scan_corpus(step):
        """Return list of match values from corpus-wide scan."""
        match_values = []

        m = 0
        while m + qlen - step <= len(corpus):
            match_values.append(_match(query, corpus[m : m-1+qlen]))
            if verbose:
                print(query, "-", corpus[m: m + qlen], _match(query, corpus[m: m + qlen]))
            m += step

        return match_values

    def index_max(v):
        """Return index of max value."""
        return max(range(len(v)), key=v.__getitem__)

    def adjust_left_right_positions():
        """Return left/right positions for best string match."""
        # bp_* is synonym for 'Best Position Left/Right' and are adjusted 
        # to optimize bmv_*
        p_l, bp_l = [pos] * 2
        p_r, bp_r = [pos + qlen] * 2

        # bmv_* are declared here in case they are untouched in optimization
        bmv_l = match_values[p_l // step]
        bmv_r = match_values[p_l // step]

        for f in range(flex):
            ll = _match(query, corpus[p_l - f: p_r])
            if ll > bmv_l:
                bmv_l = ll
                bp_l = p_l - f

            lr = _match(query, corpus[p_l + f: p_r])
            if lr > bmv_l:
                bmv_l = lr
                bp_l = p_l + f

            rl = _match(query, corpus[p_l: p_r - f])
            if rl > bmv_r:
                bmv_r = rl
                bp_r = p_r - f

            rr = _match(query, corpus[p_l: p_r + f])
            if rr > bmv_r:
                bmv_r = rr
                bp_r = p_r + f

            if verbose:
                print("\n" + str(f))
                print("ll: -- value: %f -- snippet: %s" % (ll, corpus[p_l - f: p_r]))
                print("lr: -- value: %f -- snippet: %s" % (lr, corpus[p_l + f: p_r]))
                print("rl: -- value: %f -- snippet: %s" % (rl, corpus[p_l: p_r - f]))
                print("rr: -- value: %f -- snippet: %s" % (rl, corpus[p_l: p_r + f]))

        return bp_l, bp_r, _match(query, corpus[bp_l : bp_r])

    if not case_sensitive:
        query = query.lower()
        corpus = corpus.lower()

    qlen = len(query)

    if flex >= qlen/2:
        print("Warning: flex exceeds length of query / 2. Setting to default.")
        flex = 3

    match_values = scan_corpus(step)
    pos = index_max(match_values) * step

    pos_left, pos_right, match_value = adjust_left_right_positions()

    return corpus[pos_left: pos_right].strip(), match_value

In [84]:
#Calculate for every recourse that have more than one possible match the fuzzy score. Then retain only the match with the best fuzzy score.
cancel_keys = []

for k in tqdm(ties.keys()):
  df_ties = tender_df.iloc[[matches[k][1]]]
  cos_sim = [matches[k][0]]
  for i in range(0, len(ties[k])):
    cos_sim = cos_sim + [ties[k][i][0]]
    df_ties = df_ties.append(tender_df.iloc[ties[k][i][1]])
  
  df_ties['cos_sim'] = cos_sim
  fuzzy = []
  for i in df_ties.index:
    query = df_ties.at[i, "oggetto_gara"]
    corpus = recourse_df.at[k, "recourse_object_df"]
    if len(query) <= len(corpus):
      match = get_best_match(query, corpus, step=1, flex= int(len(query) / 2)-1)
    else:
      match = get_best_match(corpus, query, step=1, flex= int(len(corpus) / 2)-1)
    
    fuzzy = fuzzy + [match[1]]
  
  df_ties['fuzzy'] = fuzzy

  df_ties = df_ties.drop(df_ties[df_ties['fuzzy'] != df_ties.fuzzy.max()].index)
  df_ties = df_ties.drop(df_ties[df_ties['cos_sim'] != df_ties.cos_sim.max()].index)
  
  if len(df_ties) == 1:
    matches[k] = [df_ties.cos_sim.tolist()[0], df_ties.index.tolist()[0]]
    cancel_keys = cancel_keys + [k]


100%|██████████| 138/138 [00:43<00:00,  3.19it/s]


In [85]:
for i in cancel_keys:
  del ties[i]

In [86]:
#Creating a dataframe with the found matches

df_matches = tender_df.iloc[[matches[0][1]]]
cos_sim = [matches[0][0]]
for i in range(1, len(matches)):
  cos_sim = cos_sim + [matches[i][0]]
  df_matches = df_matches.append(tender_df.iloc[matches[i][1]])
  
df_matches['cos_sim'] = cos_sim


result = pd.concat([recourse_df, df_matches.reset_index(drop=True)], axis=1)
result = result.sort_values('cos_sim', ascending=False)[["cig", "recourse_num", "oggetto_gara", "recourse_object_df", "cos_sim", "recourse_date_df", "anno_pubblicazione", "data_aggiudicazione_definitiva", "cf_amministrazione_appaltante", "feature_id"]].drop(index=ties.keys())

fuzzy = []

for i in tqdm(result.index):
  query = result.at[i, "oggetto_gara"]
  corpus = result.at[i, "recourse_object_df"]
  if len(query) <= len(corpus):
    match = get_best_match(query, corpus, step=1, flex= int(len(query) / 2)-1)
  else:
    match = get_best_match(corpus, query, step=1, flex= int(len(corpus) / 2)-1)
  fuzzy = fuzzy + [match[1]]
  
result['fuzzy'] = fuzzy
result = result.drop(result[result['cos_sim'] < 0.3].index)

mean = 92.23954400000001
threshold_cos_sim = 0.7
threshold_fuzzy = 0.7
result = result.drop(result[result.oggetto_gara.apply(lambda x: len(str(x)) < mean) & (result['cos_sim'] < 0.95 ) & (result['fuzzy'] < 0.90 )].index)
result = result.drop(result[(result.cos_sim < threshold_cos_sim) & (result.fuzzy < threshold_fuzzy)].index)

100%|██████████| 386/386 [00:29<00:00, 13.11it/s]


In [87]:
#Creation of a dataframe with the matches without ties and ambiguities
dataframe_matches = result.drop(result[result.duplicated(['cig'], keep=False)].index) #dropping tenders with more then 1 recourse matched (ambiguous matches)

#solve ambigous by taking the ones with best fuzzy score (if unique)
ambiguous = result[result.duplicated(['cig'], keep=False)]

solved_ambiguous = ambiguous[ambiguous['fuzzy'] == ambiguous.groupby('cig')['fuzzy'].transform('max')]
solved_ambiguous = solved_ambiguous.drop(solved_ambiguous[solved_ambiguous.duplicated(['cig'], keep=False)].index)
solved_ambiguous_list = solved_ambiguous["cig"].tolist()

#add the solved ambiguous to dataframe_matches
dataframe_matches = pd.concat([dataframe_matches,solved_ambiguous],axis=0).sort_values('cos_sim', ascending=False)
dataframe_matches

,cig,recourse_num,oggetto_gara,recourse_object_df,cos_sim,recourse_date_df,anno_pubblicazione,data_aggiudicazione_definitiva,cf_amministrazione_appaltante,feature_id,fuzzy
167,0379648F6C,201000527,GARA PER L'AFFIDAMENTO DEL SERVIZIO DI ASSISTE...,GARA PER AFFIDAMENTO SERVIZI DI ASSISTENZA TEC...,0.968949,2010-12-27,2010,2010-11-10,00169440708,NaN,0.955752
388,8397930F0E,202100054,REALIZZAZIONE E RIFACIMENTO DI ALCUNI TRATTI D...,LAVORI DI REALIZZAZIONE E RIFACIMENTO DI ALCUN...,0.968380,2021-02-19,2020,NaN,80001950940,NaN,0.989247
175,69477432E6,201700039,TRASPORTO PUBBLICO URBANO. AFFIDAMENTO ANNO 2017,TRASPORTO PUBBLICO URBANO. AFFIDAMENTO ANNO 20...,0.965957,2017-01-27,2016,2016-12-30,00071560700,NaN,1.000000
359,8803333435,202100385,AFFIDAMENTO DEI SERVIZI DI GESTIONE DEI LIVELL...,AVVERSO LA DETERMINAZIONE DI AGGIUDICAZIONE DE...,0.945680,2021-11-29,2021,NaN,CFAVCP-0000E35,NaN,0.555957
50,03738059A2,201000007,RIQUALIFICAZIONE DEL CONTESTO URBANO AI FINI T...,LAVORI DI RIQUALIFICAZIONE DEL CONTESTO URBANO...,0.942845,2010-01-11,2009,2009-12-21,00066290701,NaN,0.994536
...,...,...,...,...,...,...,...,...,...,...,...
334,5721213E4F,201500342,"PROGETTAZIONE ESECUTIVA ED ESECUZIONE, SULLA B...",APPALTO LAVORI DI VIABILITA E SERVIZI PER INSE...,0.601403,2015-09-22,2014,2015-08-05,80001790940,80001790940,0.793814
108,755451462A,201800406,INTERVENTI URGENTI DI MESSA IN SICUREZZA E MAN...,INTERVENTI URGENTI PER LA MESSA IN SICUREZZA A...,0.590222,2018-11-23,2018,NaN,82004480701,NaN,0.783626
122,015490794A,200800418,AFFIDAMENTO DEL SERVIZIO MENSA SCOLASTICA,AFFIDAMENTO SERVIZIO DI MENSA SCOLASTICA.AVVER...,0.588980,2008-11-19,2008,NaN,00067530709,NaN,0.902439
380,8022374075,202100316,FORNITURA DI ENERGIA ELETTRICA PER IL PERIODO ...,"GARA CON “IL CRITERIO DEL MINOR PREZZO”, AVENT...",0.550131,2021-10-23,2019,NaN,00249230707,NaN,0.926471


In [88]:
if not os.path.exists(dir_in + os.sep  + f"matches_{region}.csv"):
  dataframe_matches.to_csv(dir_in + os.sep  + f"matches_{region}.csv", header = True, index = False)

In [89]:
#dataframe with the ambigous matches, i.e with the tenders that were found as best match for more than one recourse.
ambiguous = ambiguous[~ambiguous.cig.isin(solved_ambiguous_list)].sort_values('cig', ascending=False)
ambiguous

,cig,recourse_num,oggetto_gara,recourse_object_df,cos_sim,recourse_date_df,anno_pubblicazione,data_aggiudicazione_definitiva,cf_amministrazione_appaltante,feature_id,fuzzy
174,73968677BE,201800205,AFFIDAMENTO DEL SERVIZIO DI CURE DOMICILIARI P...,AFFIDAMENTO DEL SERVIZIO DI CURE DOMICILIARI P...,0.799912,2018-06-18,2018,NaN,00169440708,NaN,1.000000
176,73968677BE,201800203,AFFIDAMENTO DEL SERVIZIO DI CURE DOMICILIARI P...,AFFIDAMENTO DEL SERVIZIO DI CURE DOMICILIARI P...,0.778568,2018-06-14,2018,NaN,00169440708,NaN,1.000000
131,73968677BE,201800194,AFFIDAMENTO DEL SERVIZIO DI CURE DOMICILIARI P...,ESCLUSIONE DALLA PROCEDURA DI GARA DEL COSTITU...,0.647510,2018-06-11,2018,NaN,00169440708,NaN,0.954023
401,66214777AC,201900189,LAVORI DI ADEGUAMENTO DELL'INSEDIAMENTO ABITAT...,LAVORI DI ADEGUAMENTO DELLINSEDIAMENTO ABITATI...,0.936883,2019-06-05,2016,NaN,CFAVCP-0001029,NaN,0.988372
189,66214777AC,201800046,LAVORI DI ADEGUAMENTO DELL'INSEDIAMENTO ABITAT...,GARA PER LA PROGETTAZIONE ESECUTIVA E REALIZZA...,0.840911,2018-02-06,2016,NaN,CFAVCP-0001029,NaN,0.994152
227,66214777AC,201700145,LAVORI DI ADEGUAMENTO DELL'INSEDIAMENTO ABITAT...,APPALTO RELATIVO ALLA “PROGETTAZIONE ESECUTIVA...,0.766980,2017-04-19,2016,NaN,CFAVCP-0001029,NaN,0.994152
234,6607277178,201600056,PROGETTAZIONE ESECUTIVA ED ESECUZIONE DEI LAVO...,PROGETTAZIONE ESECUTIVA ED ESECUZIONE LAVORI D...,0.717676,2016-02-25,2016,2017-05-21,00249230707,NaN,0.800000
235,6607277178,201600055,PROGETTAZIONE ESECUTIVA ED ESECUZIONE DEI LAVO...,PROGETTAZIONE ESECUTIVA ED ESECUZIONE LAVORI D...,0.717676,2016-02-25,2016,2017-05-21,00249230707,NaN,0.800000


In [90]:
if not os.path.exists(dir_in + os.sep  + f"ambiguous_{region}.csv"):
  ambiguous.to_csv(dir_in + os.sep  + f"ambiguous_{region}.csv", header = True, index = False)

In [91]:
#save the ties that were not solved.
ties_filtered = {k: v for k, v in ties.items() if ((tender_df.iloc[[matches[k][1]]].oggetto_gara.str.len()[matches[k][1]] < mean and matches[k][0] > 0.95) or ( tender_df.iloc[[matches[k][1]]].oggetto_gara.str.len()[matches[k][1]] > mean)  ) }

if not os.path.exists(dir_in + os.sep + f"ties_matches_{region}.csv"):
  for k in tqdm(ties_filtered.keys()):
    df_ties = tender_df.iloc[[matches[k][1]]]
    cos_sim = [matches[k][0]]
    for i in range(0, len(ties[k])):
      cos_sim = cos_sim + [ties[k][i][0]]
      df_ties = df_ties.append(tender_df.iloc[ties[k][i][1]])
    
    df_ties['cos_sim'] = cos_sim
    fuzzy = []
    for i in df_ties.index:
      query = df_ties.at[i, "oggetto_gara"]
      corpus = recourse_df.at[k, "recourse_object_df"]
      if len(query) <= len(corpus):
        match = get_best_match(query, corpus, step=1, flex= int(len(query) / 2)-1)
      else:
        match = get_best_match(corpus, query, step=1, flex= int(len(corpus) / 2)-1)
        
      fuzzy = fuzzy + [match[1]]
      
    df_ties['fuzzy'] = fuzzy
    df_ties["recourse_num"] = recourse_df.iloc[[k]].recourse_num.tolist()[0]
    df_ties["recourse_object_df"] = recourse_df.iloc[[k]].recourse_object_df.tolist()[0]
    df_ties["recourse_date_df"] = recourse_df.iloc[[k]].recourse_date_df.tolist()[0]
    df_ties["feature_id"] = recourse_df.iloc[[k]].feature_id.tolist()[0]
    
    if df_ties.cos_sim.max() > threshold_cos_sim or df_ties.fuzzy.max() > threshold_fuzzy:
      df_ties[["cig", "recourse_num", "oggetto_gara", "recourse_object_df", "cos_sim", "fuzzy", "data_aggiudicazione_definitiva", "cf_amministrazione_appaltante", "feature_id"]].to_csv(dir_in + os.sep + f"ties_matches_{region}.csv", header = True, index = False, mode = "a")



In [94]:
#Match filtering, to keep only the correct ones. 

x = 0.6780720949172974 #cos_sim threshold
y = 0.7578751164958062 #fuzzy threshold

#We keep a match iff cos_sim and fuzzy are both greater than the threshold or one of them is greater than 0.85
dataframe_matches_filtered = dataframe_matches.loc[ ((dataframe_matches['cos_sim'] >= x) & (dataframe_matches['fuzzy'] >= y)) | (dataframe_matches['cos_sim'] >= 0.85) | (dataframe_matches['fuzzy'] >= 0.85)]
dataframe_matches_filtered

,cig,recourse_num,oggetto_gara,recourse_object_df,cos_sim,recourse_date_df,anno_pubblicazione,data_aggiudicazione_definitiva,cf_amministrazione_appaltante,feature_id,fuzzy
167,0379648F6C,201000527,GARA PER L'AFFIDAMENTO DEL SERVIZIO DI ASSISTE...,GARA PER AFFIDAMENTO SERVIZI DI ASSISTENZA TEC...,0.968949,2010-12-27,2010,2010-11-10,00169440708,NaN,0.955752
388,8397930F0E,202100054,REALIZZAZIONE E RIFACIMENTO DI ALCUNI TRATTI D...,LAVORI DI REALIZZAZIONE E RIFACIMENTO DI ALCUN...,0.968380,2021-02-19,2020,NaN,80001950940,NaN,0.989247
175,69477432E6,201700039,TRASPORTO PUBBLICO URBANO. AFFIDAMENTO ANNO 2017,TRASPORTO PUBBLICO URBANO. AFFIDAMENTO ANNO 20...,0.965957,2017-01-27,2016,2016-12-30,00071560700,NaN,1.000000
359,8803333435,202100385,AFFIDAMENTO DEI SERVIZI DI GESTIONE DEI LIVELL...,AVVERSO LA DETERMINAZIONE DI AGGIUDICAZIONE DE...,0.945680,2021-11-29,2021,NaN,CFAVCP-0000E35,NaN,0.555957
50,03738059A2,201000007,RIQUALIFICAZIONE DEL CONTESTO URBANO AI FINI T...,LAVORI DI RIQUALIFICAZIONE DEL CONTESTO URBANO...,0.942845,2010-01-11,2009,2009-12-21,00066290701,NaN,0.994536
329,8111975589,202000073,INTERVENTO PER LA MITIGAZIONE DEL RISCHIO IDRO...,AFFIDAMENTO DEI LAVORI RELATIVI ALLINTERVENTO ...,0.911464,2020-03-10,2019,NaN,00056650948,NaN,1.000000
274,590948012B,201400463,INTERVENTI DI RISPARMIO ED EFFICIENZA ENERGETICA,AFFIDAMENTO LAVORI DI RISPARMIO ED EFFICIENZA ...,0.906623,2014-12-30,2014,2015-02-20,00170690705,NaN,0.909091
372,845853296A,202100093,LAVORI DI 'SISTEMAZIONE GEOTECNICA DELLE AREE ...,SISTEMAZIONE GEOTECNICA DELLE AREE IN FRANA DE...,0.897906,2021-03-29,2020,2021-03-23,00064560709,NaN,1.000000
48,356493663F,201200018,"SERVIZIO DI MATERIALE AFFISSIONE DI MANIFESTI,...",GARA PER AFFIDAMENTO SERVIZIO DI MATERIALE AFF...,0.885008,2012-01-26,2011,NaN,00071560700,NaN,0.857143
141,3551818CEC,201400176,APPALTO SERVIZIO MENSA SCOLASTICA,GARA DI APPALTO PER SERVIZIO MENSA SCOLASTICA,0.882571,2014-04-30,2011,NaN,00170690705,NaN,0.942857


In [95]:
if not os.path.exists(dir_in + os.sep  + f"dataframe_matches_filtered_{region}.csv"):
  dataframe_matches_filtered.to_csv(dir_in + os.sep  + f"dataframe_matches_filtered_{region}.csv", header = True, index = False)